In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
from statistics import mean
import math

In [2]:
df = pd.read_csv('CADUSD=X.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2003-09-17,0.731689,0.733353,0.729714,0.731689,0.731689,0.0
1,2003-09-18,0.732064,0.735456,0.730727,0.734484,0.734484,0.0
2,2003-09-19,0.734214,0.743108,0.733676,0.742280,0.742280,0.0
3,2003-09-22,0.745545,0.748223,0.737844,0.738443,0.738443,0.0
4,2003-09-23,0.738062,0.743052,0.736540,0.737953,0.737953,0.0


In [3]:
# df.dropna(inplace = True)
print(df.shape)
print(df.isnull().sum())
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
print(df.shape)

(4452, 7)
Date          0
Open         14
High         14
Low          14
Close        14
Adj Close    14
Volume       14
dtype: int64
(4438, 7)


In [4]:
df = df.drop("Volume", axis=1)
df.describe()
# Don't run if it's already run ...

,Open,High,Low,Close,Adj Close
count,4438.000000,4438.000000,4438.000000,4438.000000,4438.000000
mean,0.862837,0.866156,0.859548,0.862910,0.862910
std,0.099691,0.100188,0.099230,0.099679,0.099679
min,0.685857,0.689955,0.680860,0.685965,0.685965
25%,0.765185,0.767239,0.762362,0.765147,0.765147
50%,0.850629,0.853935,0.847278,0.850593,0.850593
75%,0.962630,0.966146,0.958008,0.962418,0.962418
max,1.088613,1.103509,1.071697,1.088495,1.088495


In [5]:
# SMA genarating function.. 

def addSMA(SMAcount,col):
    for i in range(len(df[col])):
        SMA = 00.0
        if i > SMAcount:
            for x in range(SMAcount+1):
                SMA += df[col][i-x]
        SMA = SMA/SMAcount
        df.at[i,'SMA '+str(SMAcount)] = SMA
    print('{} SMA genarating Completed.'.format(SMAcount))

def addpred(SMAcount,col):
    for i in range(len(df[col])-SMAcount):
        SMA = 00.0
        if i > 1:
            for x in range(SMAcount+1):
                SMA += df[col][i+x]
        SMA = SMA/SMAcount
        if SMA >= 0.5:
            SMA = 1
        else:
            SMA = 0
        df.at[i-1,'Label1'] = SMA
    df.drop([col],1)
    print('{} Label genarating Completed.'.format(SMAcount))

def addATR(n):
    for i in range(len(df['Close'])):
        atrs= []
        for x in range(n):
            atr = 0.0
            if i > 0:
                atr = df['High'][i] - df['Low'][i]
                b = abs(df['High'][i] -df['Close'][i-1] )
                a = abs(df['Low'][i] -df['Close'][i-1] )
                if atr<a:
                    atr = a
                if atr <b:
                    atr = b
            else:
                atr = df['High'][0] -df['Low'][0]
            atrs.append(atr)
            if i == x:
                break
        df.at[i,'ATR '+str(n)] = mean(atrs)
    print('Period:{} ATR genarating Completed.'.format(n))
    
def addStoch(n):
    for i in range(len(df['Close'])):
        if i>=n:
            l14 = 999999
            h14 = -999999
            for x in range(n):
                if l14>df['Low'][i-(x+1)]:
                          l14 = df['Low'][i-(x+1)]
                if h14<df['High'][i-(x+1)]:
                          h14 = df['High'][i-(x+1)]
            C = df['Close'][i]
            K = ((C-l14)*100)/(h14-l14)
            df.at[i,'Stoch '+str(n)] = K
    print('Period:{} Stoch genarating Completed.'.format(n))

def addDonchian(n):
    for i in range(len(df['Close'])):
        if i >=n:
            UC =-99999
            LC = 99999
            MID = 0
            for x in range(n):
                if UC < df['High'][i-(x+1)]:
                    UC = df['High'][i-(x+1)]
                if LC > df['Low'][i-(x+1)]:
                    LC = df['High'][i-(x+1)]
            df.at[i,'Donchian UC'+str(n)] = UC
            df.at[i,'Donchian LC'+str(n)] = LC
            df.at[i,'Donchian MID'+str(n)] = ((UC-LC)/2)
    print('Period:{} Donchian genarating Completed.'.format(n))

def addEMA(n,smooth):
    for i in range(len(df['Close'])):
        if i >= n:
            EMA = (df['Close'][i]*(smooth/(1+n))) + (df['EMA '+str(n)][i-1]*(1-(smooth/(1+n))))   
        else:
            sm = []
            for x in range(i):
                sm.append(df['Close'][x])
            if len(sm)>0:
                EMA = mean(sm)
            else:
                EMA = 0.0
        df.at[i,'EMA '+str(n)] = EMA
                
    print('Period:{} EMA genarating Completed.'.format(n))

def addMACD():  #Dont recreat the same EMA values.
    addEMA(12,2)
    addEMA(26,2)
    for i in range(len(df['Close'])):
        if i >=2:
            MACD = df['EMA 12'][i] - df['EMA 26'][i]
            df.at[i,'MACD'] = MACD
    print('MACD genarating Completed.')
def addSqrt():
    for i in range(len(df['Close'])):
        if i >= 1:
            if df['Close'][i] > df['Close'][i-1]:
                df.at[i,'Mark'] = 1
                P = math.sqrt(df['Close'][i]**2 - df['Close'][i-1]**2)
            else:
                df.at[i,'Mark'] = 0
                P = -math.sqrt(df['Close'][i-1]**2 - df['Close'][i]**2)
#             df.at[i,'Difference'] = P

def shiftCols(col,name):
    for i in range(len(df['Close'])-1):
        df.at[i,name] = df[col][i+1]
def shiftCols2(col,n):
    n = n+1
    for i in range(len(df['Close'])-n):
        i = i+n
        df.at[i,str(col) + ' '+str(n)] = df[col][i-n]
    print('{} Column shift successful'.format(col))

In [6]:
addSMA(8,'Close')
addSMA(21,'Close')
addSMA(50,'Close')
addSMA(100,'Close')
addSMA(200,'Close')
addATR(14)
addStoch(14)
# addDonchian(14)
# addMACD()
columns = df.columns[1:]
for time in range(15):
    for col in columns:
        shiftCols2(col,time)
# addSqrt()
# addpred(5,'Mark')
# shiftCols2('Close',4)
print('Preview..')
print(df.tail())

8 SMA genarating Completed.
21 SMA genarating Completed.
50 SMA genarating Completed.
100 SMA genarating Completed.
200 SMA genarating Completed.
Period:14 ATR genarating Completed.
Period:14 Stoch genarating Completed.
Open Column shift successful
High Column shift successful
Low Column shift successful
Close Column shift successful
Adj Close Column shift successful
SMA 8 Column shift successful
SMA 21 Column shift successful
SMA 50 Column shift successful
SMA 100 Column shift successful
SMA 200 Column shift successful
ATR 14 Column shift successful
Stoch 14 Column shift successful
Open Column shift successful
High Column shift successful
Low Column shift successful
Close Column shift successful
Adj Close Column shift successful
SMA 8 Column shift successful
SMA 21 Column shift successful
SMA 50 Column shift successful
SMA 100 Column shift successful
SMA 200 Column shift successful
ATR 14 Column shift successful
Stoch 14 Column shift successful
Open Column shift successful
High Column

In [7]:
print('Deleting empty rows...')
df = df[:][201:-8]
df.to_csv('dataset\datanew4.csv')
print('Done...')

Deleting empty rows...
Done...


In [8]:
df.describe()

,Open,High,Low,Close,Adj Close,SMA 8,SMA 21,SMA 50,SMA 100,SMA 200,...,Low 15,Close 15,Adj Close 15,SMA 8 15,SMA 21 15,SMA 50 15,SMA 100 15,SMA 200 15,ATR 14 15,Stoch 14 15
count,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,...,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000
mean,0.868344,0.871658,0.865064,0.868417,0.868417,0.976950,0.909713,0.885632,0.876944,0.872915,...,0.864987,0.868340,0.868340,0.976849,0.909605,0.885570,0.876960,0.870277,0.007595,51.370464
std,0.098867,0.099397,0.098369,0.098854,0.098854,0.111025,0.103160,0.100018,0.098074,0.095098,...,0.098466,0.098950,0.098950,0.111149,0.103294,0.100096,0.098053,0.108079,0.004397,33.946276
min,0.685857,0.689955,0.680860,0.685965,0.685965,0.781292,0.737565,0.724052,0.730084,0.743334,...,0.680860,0.685965,0.685965,0.781292,0.737565,0.724052,0.730084,0.000000,0.000170,-62.026663
25%,0.769556,0.772082,0.766877,0.769627,0.769627,0.865551,0.805885,0.783903,0.776796,0.773060,...,0.766877,0.769627,0.769627,0.865551,0.805885,0.783903,0.776796,0.773060,0.004589,21.481162
50%,0.860652,0.863707,0.857427,0.860807,0.860807,0.968422,0.902054,0.881125,0.871581,0.875206,...,0.857427,0.860807,0.860807,0.968422,0.902054,0.881125,0.871581,0.875206,0.006619,52.420478
75%,0.966464,0.970591,0.962807,0.966464,0.966464,1.086954,1.010679,0.983920,0.977676,0.969918,...,0.962807,0.966464,0.966464,1.086954,1.010679,0.983920,0.977676,0.969918,0.009446,80.920071
max,1.088613,1.103509,1.071697,1.088495,1.088495,1.199314,1.096746,1.057287,1.045482,1.027933,...,1.071697,1.088495,1.088495,1.199314,1.096746,1.057287,1.045482,1.027933,0.046986,149.140303
